In [1]:
import pandas as pd
import numpy as np
import random

# Test functions

In [2]:
def test(n, pCrossover, pMutation):
    sudoku_matrix = read_data()
    #print("sudoku matrix",sudoku_matrix)
    missing_values_matrix = find_missing_values(sudoku_matrix)
    #print(missing_values_matrix)
    genotypeV = genotype(missing_values_matrix)
    #print("genotipo",genotypeV)
    phenotypeV = phenotype(sudoku_matrix,genotypeV )
    #print("phenotipo",phenotypeV)
    fitness = calculate_fitness(phenotypeV)
    #print(fitness)

# Read data

In [3]:
def read_data():
    excel_file_path = 'sudoku.xlsx'
    df = pd.read_excel(excel_file_path, header=None)
    df = df.fillna(0)
    sudoku_matrix = df.values
    return sudoku_matrix

# Missing values

In [4]:
def find_missing_values(sudoku_matrix):
    # Extract the first row
    first_row = sudoku_matrix[0, :]

    # Generate a set of all possible values (1 to 9)
    all_values = set(range(1, 10))

    # Create an empty matrix to store missing values
    missing_v_matrix = np.empty((sudoku_matrix.shape[0],), dtype=object)

    # Iterate through each row and calculate missing values
    for row_idx, row in enumerate(sudoku_matrix):
        missing_values = all_values - set(row)
        missing_v_matrix[row_idx] = missing_values

    # Now 'missing_values_matrix' contains missing values for each row
    return missing_v_matrix

# Genotype

In [5]:
def genotype(missing_values_matrix):
    # Create an empty matrix to store permutations
    permutation_matrix = np.empty_like(missing_values_matrix, dtype=object)

    # Iterate through each row's missing values and generate permutations
    for row_idx, missing_values in enumerate(missing_values_matrix):
        missing_values = list(missing_values)
        permutations = np.random.permutation(missing_values)
        permutation_matrix[row_idx] = permutations

    # Now 'permutation_matrix' contains permutations for each row's missing values
    return permutation_matrix
    

# Phenotype

In [6]:
def phenotype(sudoku_matrix, genotype_matrix):
    filled_matrix = np.copy(sudoku_matrix)
    for row_idx, (sudoku_row, genotype_row) in enumerate(zip(sudoku_matrix, genotype_matrix)):
        filled_matrix[row_idx, sudoku_row == 0] = genotype_row
        
    return filled_matrix

# Fitness

In [7]:
def calculate_fitness(phenotype_matrix):
    valid_count = 0
    valid_per_row = []
    for i in range(9):
        valid_per_row_count = 0
        for j in range(9):
            value = phenotype_matrix[i, j]
            if is_valid(phenotype_matrix, i, j, value):
                valid_count += 1
                valid_per_row_count += 1
        valid_per_row.append(valid_per_row_count)
    valid_per_row_count_arr = np.array(valid_per_row)
    return valid_count, valid_per_row_count_arr

def is_valid(matrix, row, col, value):
    # Check if the value is valid in the row, column, and subgrid
    occurrences_row = np.count_nonzero(matrix[row, :] == value)
    ocurrences_colum = np.count_nonzero(matrix[:, col] == value)
    
    subgrid_row = row // 3 * 3
    subgrid_col = col // 3 * 3
    subgrid = matrix[subgrid_row:subgrid_row+3, subgrid_col:subgrid_col+3]
    
    ocurrences_grid =  np.count_nonzero(subgrid == value)
    
    if(occurrences_row >= 2 or ocurrences_colum >= 2 or ocurrences_grid >= 2 ):
        return False
    return True

# Population

In [8]:
def generatePopulation(n, sudoku_matrix, missing_values_matrix):
    p0 = []
    fitness_p0 = []
    fitness_per_row_acum = []
    for _ in range(n):
        genotypeV = genotype(missing_values_matrix)
        phenotypeV = phenotype(sudoku_matrix, genotypeV)
        fitness, fitness_per_row = calculate_fitness(phenotypeV)
        p0.append(genotypeV)
        fitness_p0.append(fitness)
        fitness_per_row_acum.append(fitness_per_row)
    p0_array = np.array(p0)
    fitness_p0_array = np.array(fitness_p0)
    fitness_per_row_acum_array = np.array(fitness_per_row_acum)
    return p0_array,fitness_p0_array, fitness_per_row_acum_array

# Elite

In [9]:
def getElite(p0, p0_fitness):
    # Find the index of the element with the highest score
    index_of_highest_score = np.argmax(p0_fitness)

    # Get the element with the highest score
    elite = p0[index_of_highest_score]
    elite_fitness = p0_fitness[index_of_highest_score]
    return np.copy(elite), np.copy(elite_fitness)

# Get Parents / Binary Tournament

In [10]:
def binaryTournament(p0,  p0_fitness):
    # Randomly select two elements
    random_indices = np.random.choice(len(p0), size=2, replace=False)
    
    #random_indices = np.random.choice(25, size=2, replace=False)
    
    random_elements = np.copy(p0[random_indices])
    random_scores = np.copy(p0_fitness[random_indices])

    # Find the index of the element with the highest score
    index_of_highest_score = np.argmax(random_scores)

    # Get the element with the highest score
    element_with_highest_score = np.copy(p0[index_of_highest_score])
    highest_score = np.copy(p0_fitness[index_of_highest_score])
    
    return index_of_highest_score, element_with_highest_score, highest_score

def getParents(p0, p0_fitness):
    idx1 = idx2 = 0
    idx1, p1, hs1 = binaryTournament(p0,  p0_fitness)
    idx2, p2, hs2 = binaryTournament(p0,  p0_fitness)
    if(idx2 == idx1):
        idx2 = idx1+1
        p2 = p0[idx2]
        hs2 = p0_fitness[idx2]
    return p1,p2, hs1, hs2, idx1, idx2


# Mutate 

In [11]:
def mutate(new_individual):
    # Select one of the arrays from the array of arrays
    selected_array_index = np.random.randint(0, len(new_individual))
    integer_array = np.copy(new_individual[selected_array_index])

    # Choose two distinct random indices
    indices_to_swap = np.random.choice(len(integer_array), size=2, replace=False)

    # Swap the elements at the selected indices
    integer_array[indices_to_swap[0]], integer_array[indices_to_swap[1]] = (
        integer_array[indices_to_swap[1]],
        integer_array[indices_to_swap[0]]
    )
    
    new_individual[selected_array_index] = integer_array
    
    return np.copy(new_individual)

# Crossover

In [12]:
def crossover(p1, p2, hs1, hs2,dominancePerc):
    # Calculate the number of elements to choose from each array
    random_integer = random.randint(1, 8)
    new_individual = np.concatenate((p1[:random_integer], p2[random_integer:]))

    return new_individual

def crossover1(p1, p2, hs1, hs2,idx1, idx2, dominancePerc,new_individuals_fitness_p_row):
    # Calculate the number of elements to choose from each array
    
    new_individual = []
    scores_array_p1 = new_individuals_fitness_p_row[idx1]
    scores_array_p2 = new_individuals_fitness_p_row[idx2]
    for i in range(len(p1)):
        score_p1 = scores_array_p1[i]  # Score of p1's element i
        score_p2 = scores_array_p2[i]  # Score of p2's element i

        if score_p1 > score_p2:
            selected_element = p1[i]
        else:
            selected_element = p2[i]

        new_individual.append(np.copy(selected_element))

    return np.array(new_individual, dtype=object)


# Solve

In [15]:
def solve(n, pCrossover, pMutation, dominancePerc):
    sudoku_matrix = read_data()
    missing_values_matrix = find_missing_values(sudoku_matrix)
    p0, p0_fitness, p0_fitness_per_row = generatePopulation(n, sudoku_matrix, missing_values_matrix)

    elite, elite_fitness = getElite(p0, p0_fitness)
    k = 0
    equal_elite = 0
    prev_elite_fitness = elite_fitness
    pMutationOrig = pMutation
    while elite_fitness < 81 and k <= 50000:
        if elite_fitness == prev_elite_fitness:
            equal_elite+=1
        else:
            equal_elite=0
            pMutation = pMutationOrig
        if equal_elite == 50:
            pMutation = 1
        new_individuals = []
        new_individuals_fitness = []
        new_individuals_fitness_p_row = []
        
        k = k+1
        #p0_fitness_copy = np.copy(p0_fitness)
        #sorted_indices = np.argsort(p0_fitness_copy)[::-1]
        #p0_sorted = [p0[i] for i in sorted_indices]
        #p0 = np.copy(p0_sorted)
        #p0_fitness = p0_fitness_copy[sorted_indices]
        for i in range(n):
            
            random_p = random.random()

            #idx, new_individual, hs =  binaryTournament(p0,  p0_fitness)
            new_individual= p0[0]
            
            if random_p <= pCrossover:
                p1, p2, hs1, hs2, idx1, idx2 = getParents(p0, p0_fitness)
                #new_individual = crossover(p1, p2, hs1, hs2,idx1, idx2, dominancePerc,p0_fitness_per_row)
                new_individual = crossover(p1, p2, hs1, hs2,dominancePerc)
            #random_p = random.random()
            if random_p <= pMutation:
                new_individual = mutate(new_individual)
 

            new_individuals.append(new_individual)
            fitness, fit_p_row = calculate_fitness(  phenotype(sudoku_matrix, new_individual) )
            new_individuals_fitness.append(fitness)
            new_individuals_fitness_p_row.append(fit_p_row)

        p0 = np.copy(np.array(new_individuals))
        p0_fitness = np.copy(np.array(new_individuals_fitness))
        p0_fitness_per_row = np.copy(np.array(new_individuals_fitness_p_row))
        new_elite, new_elite_fitness = getElite(p0, p0_fitness)
        if(new_elite_fitness > elite_fitness):
            elite_fitness = new_elite_fitness
            elite = new_elite
        print("K: ",k, "elite fitness: ", elite_fitness)
    return elite, elite_fitness

# Main

In [16]:
elite, elite_fitness = solve(50, 0.5, 0.5, 0.7)
print(elite, elite_fitness)

K:  1 elite fitness:  31
K:  2 elite fitness:  31
K:  3 elite fitness:  31
K:  4 elite fitness:  31
K:  5 elite fitness:  31
K:  6 elite fitness:  31
K:  7 elite fitness:  31
K:  8 elite fitness:  31
K:  9 elite fitness:  31
K:  10 elite fitness:  31
K:  11 elite fitness:  31
K:  12 elite fitness:  31
K:  13 elite fitness:  31
K:  14 elite fitness:  31
K:  15 elite fitness:  31
K:  16 elite fitness:  31
K:  17 elite fitness:  31
K:  18 elite fitness:  31
K:  19 elite fitness:  31
K:  20 elite fitness:  31
K:  21 elite fitness:  31
K:  22 elite fitness:  31
K:  23 elite fitness:  31
K:  24 elite fitness:  31
K:  25 elite fitness:  31
K:  26 elite fitness:  31
K:  27 elite fitness:  31
K:  28 elite fitness:  31
K:  29 elite fitness:  31
K:  30 elite fitness:  31
K:  31 elite fitness:  31
K:  32 elite fitness:  31
K:  33 elite fitness:  31
K:  34 elite fitness:  31
K:  35 elite fitness:  31
K:  36 elite fitness:  31
K:  37 elite fitness:  31
K:  38 elite fitness:  31
K:  39 elite fitness:

KeyboardInterrupt: 

In [ ]:
print(np.random.choice(10, size=11, replace=False))

In [ ]:
sudoku_matrix = read_data()
total, divided = calculate_fitness(sudoku_matrix)
print(divided[8])
print(total, divided)
print(sudoku_matrix)

In [ ]:
#mejoras
# 1. offspring by row
# 2. sort parents and always choose the ones with highest scores

In [ ]:
print(p0_fitness)

In [ ]:
# Example p0 array and p0_fitness_array
p0 = [np.array([1, 2, 3]), np.array([4, 5, 6]), np.array([7, 8, 9])]
p0_fitness_array = np.array([0.8, 0.5, 0.9])

# Sort both arrays based on p0_fitness_array in descending order
sorted_indices = np.argsort(p0_fitness_array)[::-1]
sorted_p0 = [p0[i] for i in sorted_indices]
sorted_p0_fitness_array = p0_fitness_array[sorted_indices]

print("Sorted p0 array:", sorted_p0)
print("Sorted p0_fitness_array:", sorted_p0_fitness_array)

In [ ]:
pp = [1,2,3,4,5]
pp[:4]